In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

import lightning as L

In [ ]:
'''
This is a small test training data for the model to test functionality, not accuracy
'''

input_token_to_id = {

}

input_id_to_token = dict(map.reversed(input_token_to_id.items()))

output_token_to_id = {

}

output_id_to_token = dict(map.reversed(output_token_to_id.items()))

inputs = torch.tensor([

])

labels = torch.tensor([
    
])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model=1, max_len=5000):
        super().__init__()

        pe = torch.zeros(max_len, d_model)

        position = torch.arange(0, max_len).float().unsqueeze(1)
        embedding_index = torch.arange(0, d_model, 2).float()

        div_term = 1/torch.tensor(10000).pow(embedding_index/d_model)

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe)
    
    def forward(self, word_embeddings):
        return word_embeddings + self.pe[:word_embeddings.size(0), :]

In [ ]:
class Attention(nn.Module):
    def __init__(self, d_model):
        super().__init__()

        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        self.row_dim = 0
        self.col_dim = 1
    
    def forward(self, encodings_q, encodings_k, encodings_v, mask=None):
        q = self.W_q(encodings_q)
        k = self.W_k(encodings_k)
        v = self.W_v(encodings_v)

        sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))

        scaled_sims = sims/torch.tensor(k.size(self.col_dim)**0.5)

        if mask is None:
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)
        
        attention_percents = F.softmax(scaled_sims, dim=self.col_dim)
        attention_scores = torch.matmul(attention_percents, v)

        return attention_scores

In [ ]:
class EncoderTransformer(L.LightningModule):
    def __init__(self, num_tokens, d_model, max_len=5000):
        super().__init__()

        self.we = nn.Embedding(num_embeddings=num_tokens, embedding_dim=d_model)
        self.pe = PositionalEncoding(d_model=d_model, max_len=max_len)
        self.self_attention = Attention(d_model=d_model)
        self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)

        self.loss = nn.CrossEntropyLoss()

    def forward(self, token_ids):
        word_embeddings = self.we(token_ids)
        position_encoded = self.pe(word_embeddings)

        